#### INFO


***INITIAL SETUP***

1. *Runtime ->  Change Runtime Type -> Python 3 + GPU*
2. *Fill in `GDRIVE_DIR` in [code #5] (name of the folder in gdrive that contains your code and data)*

***RUNNING THE SCRIPTS***
1. *Runtime -> Run All* before the first run. 
2. Pay attention to the authentification. Also, sometimes so you may need to clean the drive cache ([code #5])
4. Training - when re-running the training, you **do not** have to run any of the preceding scripts. However, Clearing google drive cache might help you.

***ABOUT***

- *[code #1] Install dependencies for google authentification*
- *[code #2] Allow collaboratory to read from & write into your google drive"*
- *[code #3] Download tensorboard accesories*
- *[code #4] Setup paths for tensorboard and your script. *
- *[code #5] Initialize the functions for tensorflow and runner function*
- *[code #6] Mounts the google drive to current FS as ./drive.*
- *[code #7] VM reset script - Kills all processes. Sometimes helps, once the GPU tensorflow stops working, drive fails etc. **! RESETS AUTH !** *
- *[code #8] Sample call to run function that actually runs tensorboard and your script.*


***FINAL NOTES***

Most of this script was copy-pasted from https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d

The Tensorflow part is taken from https://github.com/mixuala/colab_utils.git + Special thanks to Štěpán Hojdar.

Feel free to improve this script - just send me suggestions / bug reports / improved scripts and I will upload it here.

Created by Vladislav Vancák & Petr Houška.

#### SETUP

In [0]:
# DRIVE & AUTH DEPENDENCIES
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# TENSORBOARD DEPENDENCIES
!git clone https://github.com/mixuala/colab_utils.git

In [0]:
# GOOGLE AUTH
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

ROOT = None

#### CONFIG

In [0]:
# SCRIPT PROPERTIES !!! CHANGE GOOGLE DRIVE FOLDER HERE e.g. GDRIVE_DIR="MFF/DL/mnist" !!!
import os

GDRIVE_DIR="<FOLDER_NAME_IN_GDRIVE>"

if ROOT is not None:
  os.chdir(ROOT)

ROOT = %pwd
SCRIPT_PATH = os.path.join(ROOT, 'drive', GDRIVE_DIR)
LOG_DIR = os.path.join(SCRIPT_PATH, 'logs')

In [0]:
# RUNNER FUNCTION
import tensorflow as tf
def run_script(name, parameters):
  SCRIPT_FILE= name
  SCRIPT_ARGS= parameters

  os.chdir(SCRIPT_PATH)

  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    print('[WARN]: GPU device not found. Running in CPU mode')
  else:
    print('[INFO]: Found GPU at: {}'.format(device_name))  
  print('<<= <=> =>>')
  
  %run $SCRIPT_FILE $SCRIPT_ARGS

# TENSORBOARD
def tensorboard():
  
  import os
  import sys
  import colab_utils.tboard
  
  os.chdir(ROOT)
  colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR)

# GOOGLE DRIVE MOUNT / CLEAR CACHE
def google_drive():
  os.chdir(ROOT)
  
  if os.path.exists('drive'):
    !google-drive-ocamlfuse -cc

  else:
    !mkdir -p drive
    !google-drive-ocamlfuse drive

# RUN ALL
def run(name, parameters):
  google_drive()
  print('<<= <=> =>>')

  tensorboard()
  print('<<= <=> =>>')
  
  run_script(name, parameters)

#### UTILS

In [0]:
# VM RESET !!! RESETS GOOGLE AUTH !!
# !kill -9 -1 

#### RUN

In [0]:
run(
    "<script_name>",
    "<script_parameters>"
)